In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os.path as osp
import itertools
import copy
import uuid
import pickle


import torch
import torch.nn.functional as F
import torch_geometric
import numpy as np

import torch.nn as nn
import torch.optim as optim

import chofer_torchex.pershom as pershom

from torch.nn import Sequential, Linear, ReLU
from torch.optim.lr_scheduler import MultiStepLR

from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GINConv, global_add_pool

from chofer_torchex import pershom
ph = pershom.pershom_backend.__C.VertFiltCompCuda__vert_filt_persistence_batch

from chofer_torchex.nn import SLayerRationalHat
from collections import defaultdict, Counter

device = 'cuda:1'

import core.model
from core.data import dataset_factory
from core.utils import my_collate, evaluate
from core.train_engine import *

from tensorboardX import SummaryWriter



Using /tmp/chofer_torch_extensions as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /tmp/chofer_torch_extensions/pershom_cuda_ext/build.ninja...
Building extension module pershom_cuda_ext...
Loading extension module pershom_cuda_ext...


In [3]:
dataset_names = [
#         'COLLAB',
#         'IMDB-MULTI',
#         'IMDB-BINARY',
#         'ENZYMES',
        #'PTC_PGNN',
        #'PTC_FM',
        #'PTC_FR',
        #'PTC_MM',
        #'PTC_MR',
#         'PROTEINS',
#         'DD',
        'NCI1',
        #'MUTAG'
]

dataset_has_node_lab = {n: dataset_factory(n, verbose=False).num_node_lab is not None for n in dataset_names}
dataset_has_node_lab

{'NCI1': True}

In [4]:
training_cfg = {
    'lr': 0.01, 
    'lr_drop_fact': 0.5, 
    'num_epochs': 100, 
    'epoch_step': 20,
    'batch_size': 64,
    'weight_decay': 10e-06,
    'validation_ratio': 0.1
}
training_cfgs = [training_cfg]

In [5]:
proto = {
    'model_type': 'PershomRigidDegreeFilt',
    'use_super_level_set_filtration': None, 
    'num_struct_elements': 100, 
}
model_cfgs_PershomRigidDegreeFilt = []
for b in [False, True]:
    tmp = copy.deepcopy(proto)
    
    tmp['use_super_level_set_filtration'] = b
    
    model_cfgs_PershomRigidDegreeFilt.append(tmp)
    
len(model_cfgs_PershomRigidDegreeFilt)

2

In [6]:
proto = {
    'model_type': 'PershomLearnedFilt',
    'use_super_level_set_filtration': None, 
    'use_node_degree': None, 
    'use_node_label': None, 
    'gin_number': 1, 
    'gin_dimension': 64,
    'gin_mlp_type': 'lin_bn_lrelu_lin', 
    'num_struct_elements': 100, 
}
model_cfgs_PershomLearnedFilt = []

B = [(True, True), (False, True), (True, False)]

for (a, b), c in itertools.product(B, [True, False]):
    tmp = copy.deepcopy(proto)

    tmp['use_node_degree'] = a
    tmp['use_node_label'] = b
    tmp['use_super_level_set_filtration'] = c    

    model_cfgs_PershomLearnedFilt.append(tmp)
    
len(model_cfgs_PershomLearnedFilt)

6

In [7]:
proto = {
    'model_type': 'GIN',
    'use_node_degree': None, 
    'use_node_label': None, 
    'gin_number': 5, 
    'gin_dimension': 64,
    'gin_mlp_type': 'lin_bn_lrelu_lin', 
}
model_cfgs_GIN = []

B = [(True, True), (False, True), (True, False)]

for a, b in B:
    tmp = copy.deepcopy(proto)

    tmp['use_node_degree'] = a
    tmp['use_node_label'] = b

    model_cfgs_GIN.append(tmp)
    
len(model_cfgs_GIN)

3

In [8]:
def combine(dataset_names, training_cfgs, model_cfgs):
    exp_cfgs = []
    continued = 0
    for a, b, c in itertools.product(dataset_names, training_cfgs, model_cfgs):

        # filter out datasets which have no node labels
        ds_has_node_lab = dataset_has_node_lab[a]

        if 'use_node_label' in c:
            use_node_lab = c['use_node_label']

            if (not ds_has_node_lab) and use_node_lab:
#                 print(a, c['model_type'])
                continue

        tmp = {
            'dataset_name': a, 
            'training': b, 
            'model': c
        }
        exp_cfgs.append(tmp)
        
    return exp_cfgs

def write_file(dataset_names, training_cfgs, model_cfgs, output_dir):
    exp_cfgs = combine(dataset_names, training_cfgs, model_cfgs)
    file_name = "exp_cfgs__" + "_".join(dataset_names) + ".json"
    with open(os.path.join(output_dir, file_name), 'w') as fid:
        json.dump(exp_cfgs, fid)
        
    print('Num cfgs: ', len(exp_cfgs))

In [9]:
output_dir = '/home/pma/chofer/repositories/nips_2019_code/results'

write_file(dataset_names, training_cfgs, model_cfgs_PershomLearnedFilt, output_dir)

Num cfgs:  6


In [10]:
exp_cfgs = combine(dataset_names, 
                   training_cfgs,
                   model_cfgs_PershomRigidDegreeFilt)
print(len(exp_cfgs))
with open(os.path.join(output_dir, 'exp_cfgs_rigid_degree_filtration.json'), 'w') as fid:
    json.dump(exp_cfgs, fid)
    
    
exp_cfgs = combine(dataset_names, 
                   training_cfgs,
                   model_cfgs_PershomLearnedFilt)
print(len(exp_cfgs))
with open(os.path.join(output_dir, 'exp_cfgs_learnt_filtration.json'), 'w') as fid:
    json.dump(exp_cfgs, fid)

    
exp_cfgs = combine(dataset_names, 
                   training_cfgs,
                   model_cfgs_GIN)
print(len(exp_cfgs))
with open(os.path.join(output_dir, 'exp_cfgs_learnt_filtration.json'), 'w') as fid:
    json.dump(exp_cfgs, fid)

6
10
5
